# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd


In [2]:
bank_data = pd.read_csv('/datasets/data.csv') #Прочитаем файл *data.csv* и сохраним его в переменной *bank_data*.

In [3]:
bank_data.info() #Рассмотрим общую информацию о данных таблицы *bank_data*. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
bank_data.shape


(21525, 12)

**Мы видим, что в таблице всего 12 столбцов:**

* children — количество детей в семье

* days_employed — общий трудовой стаж в днях

* dob_years — возраст клиента в годах

* education — уровень образования клиента

* education_id — идентификатор уровня образования

* family_status — семейное положение

* family_status_id — идентификатор семейного положения

* gender — пол клиента

* income_type — тип занятости

* debt — имел ли задолженность по возврату кредитов

* total_income — ежемесячный доход

* purpose — цель получения кредита



**- Количество значений в некоторых столбцах различается, что свидетельствует о том, что в данных есть пропущенные значения.**

**- Тип float в столбцах days_employed и total_income может затруднить работу с данными.**


In [5]:
bank_data.head(10) #Посмотрим первые 10 записей таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Бросаются в глаза отрицательные значения столбца *days_employed*. Будем исходить из того, что по каким-либо причинам эти данные были отображены со знаком минус.

**На всякий случай посмотрим каждый столбец:**

In [6]:
bank_data.groupby('children')['children'].count()#Проверим на аномалии количество детей методом count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

**Два подпункта не могут соответствовать действительности:**
        
**-1** - количество детей не может быть отрицательным;

**20** - теоретически это возможно, но необходимо руководствоваться здравым смыслом


In [7]:
bank_data.groupby('dob_years')['dob_years'].count() #Посмотрим, есть ли необычные значения возраста

dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

**Возраст 0 лет не может соответствовать действительности.**

In [8]:
bank_data.groupby('education')['education'].count()#Подсчитаем количество каждой категории в education методом count()

education
ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64

 **Анализ данных затруднителен, поэтому требуется перевести все в единый регистр.**

In [9]:
bank_data.groupby('education_id')['education_id'].count()

education_id
0     5260
1    15233
2      744
3      282
4        6
Name: education_id, dtype: int64

In [10]:
bank_data.groupby('family_status')['family_status'].count()

family_status
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
гражданский брак          4177
женат / замужем          12380
Name: family_status, dtype: int64

In [11]:
bank_data.groupby('family_status_id')['family_status_id'].count()

family_status_id
0    12380
1     4177
2      960
3     1195
4     2813
Name: family_status_id, dtype: int64

In [12]:
bank_data.groupby('gender')['gender'].count()

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

**Видим одно значение "XNA". Возможно, ошиблись при заполнении анкеты.**

In [13]:
bank_data.groupby('income_type')['income_type'].count()

income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: income_type, dtype: int64

In [14]:
bank_data.groupby('debt')['debt'].count() 

debt
0    19784
1     1741
Name: debt, dtype: int64

**Видим, что большинство клиентов исполняют вовремя свои обязательства.**

In [15]:
bank_data.groupby('purpose')['purpose'].count() #Посмотрим, какие цели у клиентов для получения кредита  

purpose
автомобили                                478
автомобиль                                495
высшее образование                        453
дополнительное образование                462
жилье                                     647
заняться высшим образованием              496
заняться образованием                     412
на покупку автомобиля                     472
на покупку подержанного автомобиля        479
на покупку своего автомобиля              505
на проведение свадьбы                     777
недвижимость                              634
образование                               447
операции с жильем                         653
операции с коммерческой недвижимостью     651
операции с недвижимостью                  676
операции со своей недвижимостью           630
покупка жилой недвижимости                607
покупка жилья                             647
покупка жилья для сдачи                   653
покупка жилья для семьи                   641
покупка коммерческой недви

**Чтобы избежать расплывчатых формулировок целей клиентов для получения кредита, разделим список на несколько категорий:**

* Автомобиль

* Недвижимость

* Образование

* Ремонт жилья

* Свадьба



**Вывод**

1) Основная часть данных смотрится правдоподобной;

2) Мы можем игнорировать столбцы days_employed, dob_years, education,education_id и gender, исходя из поставленной задачи


## Шаг 2. Предобработка данных

### Обработка пропусков

Произведем замену отричательных значений столбца days_employed на абсолютное методом abs():

In [16]:
bank_data['days_employed'] = bank_data['days_employed'].abs() 

In [17]:
bank_data.isnull().sum() #Удостоверимся о наличии пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Одинаковое количество пропущенных значений в days_employed и total_income. Возможно, они пропущены в одинаковых наблюдениях.

Общий трудовой стаж в днях не составляет особой важности в данном исследовании, а доход оказывает влияние на результаты. Найдем  медиану median().

In [18]:
#days_employed_median = bank_data['days_employed'].median()
#days_employed_median

In [19]:
#Заполним значением медианы пропуски для данного столбца. 

In [20]:
#bank_data['days_employed'] = bank_data['days_employed'].fillna(days_employed_median)

Отобразим в переменной classes_id все уникальные значения столбца income_type. После этого с помощью цикла исследуем все значения classes_id, найдем медиану для каждой категории.

In [21]:
classes_id = bank_data['income_type'].unique()

In [22]:
for item in classes_id:
    median = bank_data[(bank_data['income_type'] == item)]['days_employed'].median()
    print('|{: <15}|{: <10}{: <18}|'.format(item,'медиана =',median))

|сотрудник      |медиана = 1574.2028211070851|
|пенсионер      |медиана = 365213.30626573117|
|компаньон      |медиана = 1547.3822226779334|
|госслужащий    |медиана = 2689.3683533043886|
|безработный    |медиана = 366413.65274420456|
|предприниматель|медиана = 520.8480834953765 |
|студент        |медиана = 578.7515535382181 |
|в декрете      |медиана = 3296.7599620220594|


Перезапишем пустые значение для каждой категории на медиану, затем проверим результаты.

In [23]:
for item in classes_id:
    category =  bank_data[(bank_data['income_type'] == item)]
    median = category['days_employed'].median()
    bank_data[(bank_data['income_type'] == item)] = bank_data[(bank_data['income_type'] == item)].fillna(median)

Произведем проверку на пропуски: 

In [24]:
bank_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [25]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Остались пропуски только в столбце total_income. Подсчитаем количество каждой категории в income_type методом count() и выведем 10 последних строк методом tail().

In [26]:
bank_data.groupby('income_type')['total_income'].count()

income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: total_income, dtype: int64

In [27]:
bank_data.sort_values(by='total_income').tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11071,1,1851.200013,36,высшее,0,гражданский брак,1,F,сотрудник,0,1.286281e+06,покупка коммерческой недвижимости
15268,1,10207.448165,64,высшее,0,в разводе,3,M,компаньон,0,1.350246e+06,жилье
18353,1,3173.282035,41,высшее,0,Не женат / не замужем,4,F,компаньон,0,1.427934e+06,автомобиль
18368,1,333.935516,41,ВЫСШЕЕ,0,гражданский брак,1,M,компаньон,0,1.551153e+06,свадьба
17503,0,2285.476482,43,среднее,1,женат / замужем,0,M,компаньон,0,1.597613e+06,операции с недвижимостью
17178,0,5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
20809,0,4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
9169,1,5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
19606,1,2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
12412,0,1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью


Отобразим в переменной classes все уникальные значения столбца income_type. После этого с помощью цикла исследуем все значения classes, найдем медиану для каждой категории.

In [28]:
classes = bank_data['income_type'].unique()

In [29]:
for item in classes:
    median = bank_data[(bank_data['income_type'] == item)]['total_income'].median()
    print('|{: <15}|{: <10}{: <20}|'.format(item,'медиана =',median))

|сотрудник      |медиана = 133546.4572377553   |
|пенсионер      |медиана = 128747.67556966442  |
|компаньон      |медиана = 162401.35155456822  |
|госслужащий    |медиана = 139034.45238630794  |
|безработный    |медиана = 131339.7516762103   |
|предприниматель|медиана = 249841.99651529055  |
|студент        |медиана = 98201.62531401133   |
|в декрете      |медиана = 53829.13072905995   |


Перезапишем пустые значение для каждой категории на медиану, затем проверим результаты.


In [30]:
for item in classes:
    category =  bank_data[(bank_data['income_type'] == item)]
    median = category['total_income'].median()
    bank_data[(bank_data['income_type'] == item)] = bank_data[(bank_data['income_type'] == item)].fillna(median)

In [31]:
bank_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [32]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**



Вероятно, пропуски произошли по следующим причинам:

1) Банку не предоставили данные;

2) Возникновение ошибки во время выгрузки;

Произвели замену нулевых строк на медианное значение в столбце "dob_years".



### Обработка аномальных значений

Помним, что при просмотре данных мы увидели аномальные значения количества детей: -1 и 20. Вероятно, их ввели по ошибке.
Сумма двух значений очень мала, поэтому заменим эти значения на более вероятные: 1 и 2:

In [33]:
bank_data['children'] = bank_data['children'].replace(20,2).abs()

In [34]:
bank_data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

1) Аномальные значения заменены на наиболее вероятные (в случае с возрастом взяли медианное значение);

2) Возможно, ошибки связаны с человеческим фактором (дефис отобразился как знак "минус")

### Замена типа данных

Необходимо заменить тип float64 в столбцах days_employed и total_income на целочисленные значения int64:


In [35]:
bank_data['days_employed'] = bank_data['days_employed'].astype('int')

In [36]:
bank_data['total_income'] = bank_data['total_income'].astype('int')

In [37]:
bank_data.info() #Проверим результаты

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**


Для комфортной работы с данными заменили тип float в столбцах days_employed и total_income на  int64 методом astype.

### Обработка дубликатов

Проверим наличие дубликатов методом duplicated, если есть посмотрим на их количество методом sum:

In [38]:
bank_data.duplicated().sum()

54

Помним, что в столбце education из-за регистра образовалось несколько  одинаковых категорий образования. 
Применим метод str.lower() в Pandas и проверим результаты:

In [39]:
bank_data['education'] = bank_data['education'].str.lower()


In [40]:
bank_data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Снова проверим наличие дубликатов методом duplicated:

In [41]:
bank_data.duplicated().sum()

71

После перевода его данных к единому регистру, количество дубликатов увеличилось с 55 до 72.
Количество дубликатов меньше 1% данных, на результаты иследования это не повлияет. 
Применим метод drop_duplicates(), сбросим индексацию строк reset_index() и удалим старую (drop= True):

In [42]:
bank_data = bank_data.drop_duplicates().reset_index(drop= True)

Проверим результат:

In [43]:
bank_data.duplicated().sum()

0

**Вывод**

Общее количество дубликатов составляет менее 1%, что никак не повлияет на результат исследования, поэтому их удалили.
Вероятно, дубликаты образовались из-за ошибок работников банка, либо из-за неполадок с базой данных.

### Лемматизация
Функция лемматизации уже импортирована заранее.

В столбце purpose обнаружены фразы, записанные в разных формах. Проведем лемматизацию:

In [44]:
#Так как у нас нет запросов со словами, имеющими одинаковые корни/основу, поэтому можно обойтись без специальных функций:
def lemma(row):
    row=row.lower()
    if 'ремонт' in row:
        return 'ремонт жилья'
    if 'жил' in row:
        return 'недвижимость'
    if 'недвиж' in row:
        return 'недвижимость'
    if 'автомоб' in row:
        return 'автомобиль'
    if 'образ' in row:
        return 'образование'
    if 'свад' in row:
        return 'свадьба'
    else:
        return 'no_info'
bank_data['purpose']=bank_data['purpose'].apply(lemma)
bank_data.groupby('purpose')['purpose'].agg(['count'])

,count
purpose,
автомобиль,4306
недвижимость,10204
образование,4013
ремонт жилья,607
свадьба,2324


**Вывод**

После лемматизации мы получили 5 категорий. Наибольшая группа по цели получения кредита оказалась в категории недвижимости, т.к.эта категория включила в себя постройку и приобретение разных объектов. Наименьшая группа оказалась по ремонту жилья.



### Категоризация данных

Разобьем людей по категориям доходов следующим образом:

1) очень высокий - группа людей, превышающими медиану в несколько раз;

2) высокий - люди с доходами, ощутимо превышающими медиану;

3) средний - диапазон (0.75;2] медианы;

4) низкий - 0.75 от медианного дохода;

5) no_information -группа людей, о которых нет информации


In [45]:
bank_data_income_median=bank_data['total_income'].median()
print(bank_data_income_median)


138975.0


In [46]:
bank_data_income_median=bank_data['total_income'].median()
def category(income):
    if income>4*bank_data_income_median:
        return 'очень высокий'
    if income>2*bank_data_income_median:
        return 'высокий'
    if income>0.75*bank_data_income_median:
        return 'средний'
    if income<=0.75*bank_data_income_median:
        return 'низкий' 
    else:
        return 'no_information'

bank_data['category_total_income']=bank_data['total_income'].apply(category)
bank_data.groupby('category_total_income')['category_total_income'].agg(['count'])
bank_data.drop(['total_income'], axis='columns',inplace=True)
bank_data=bank_data.reset_index(drop=True)
display(bank_data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,purpose,category_total_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,недвижимость,средний
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,образование,средний
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,свадьба,средний
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,недвижимость,средний
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,автомобиль,средний
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,недвижимость,низкий
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,автомобиль,средний


**Вывод**

Обновленная таблица теперь содержит 6 столбцов: category_total_income (категория величины доходов), purpose (цели кредита),
debt(наличие задолжности по кредиту), income_type (тип занятости), family_status (семейное положение) и children (количество детей).


##  Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

В зависимости от числа детей в семье построим сводную таблицу должников:

In [47]:
ch_debt=bank_data.groupby('children')['debt'].agg(['count','sum','mean'])
ch_debt.columns=['общее кол-во','кол-во должников','доля должников']
display(ch_debt)

,общее кол-во,кол-во должников,доля должников
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод**


1) Очень вероятно, что отсутствие должников среди клиентов с 5 детьми случайность, т.к. о них имеется небольшое количество данных;

2) Наибольший процент должников среди родителей с 4-мя детьми тоже, скорее всего, случайность;

3) Наличие хотя бы одного ребенка в семье увеличивает риск просрочки кредита.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

В зависимости от семейного положения построим сводную таблицу должников:

In [48]:
fs_debt=bank_data.groupby('family_status')['debt'].agg(['count','sum','mean'])
fs_debt.columns=['общее кол-во','кол-во должников','доля должников']
display(fs_debt)

,общее кол-во,кол-во должников,доля должников
family_status,,,
Не женат / не замужем,2810,274,0.097509
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452


**Вывод**

1) Наличие супруга/супруги не влияет на возвратность кредита;

2) Состоящие в гражданском браке оказываются должниками чаще, чем люди, которые зарегистрировали официально свои отношения.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
cti_debt=bank_data.groupby('category_total_income')['debt'].agg(['count','sum','mean'])
cti_debt.columns=['общее кол-во','кол-во должников','доля должников']
display(cti_debt)

,общее кол-во,кол-во должников,доля должников
category_total_income,,,
высокий,2181,161,0.073819
низкий,6684,526,0.078695
очень высокий,146,9,0.061644
средний,12443,1045,0.083983


**Вывод**

1) Судя по данной таблице, люди со очень высокими доходами очень редко оказываются в должниках. Но об этом нельзя утверждать с полной уверенностью, т.к. данных о них немного;

2) Однозначный вывод можно сделать о том, что люди с высокими и низкими доходами более дисциплинированы, чем люди со средним доходом. 

- Как разные цели кредита влияют на его возврат в срок?

В зависимости от целей получения кредита построим сводную таблицу должников:

In [50]:
purp_debt=bank_data.groupby('purpose')['debt'].agg(['count','sum','mean'])
purp_debt.columns=['общее кол-во','кол-во должников','доля должников']
display(purp_debt)

,общее кол-во,кол-во должников,доля должников
purpose,,,
автомобиль,4306,403,0.093590
недвижимость,10204,747,0.073207
образование,4013,370,0.092200
ремонт жилья,607,35,0.057661
свадьба,2324,186,0.080034


**Вывод**


1) Выплаты по кредитам за автомобиль и образование задерживают чаще, чем на другие цели;

2) Некоторые категории (недвижимость и ремонт жилья) более надежны, вероятно это связано с тщательной проверкой клиента.

## Общий вывод

В ходе работы над данными банка мы обнаружили ошибки: огромные и отрицательные значения в столбце трудового стажа, слишком много пропущенных значений в столбцах ежемесячного дохода, некоторые данные никаким образом не соответствуют действительности. По некоторым пунктам необходимо запросить информацию у кредитного отдела.

После преобразования и детального анализа данных (заполнения пропусков, перевода некоторых подпунктов в единый регистр, формулировки целей получения кредита, подготовки сводных таблиц) мы сделали некоторые выводы:

- Наличие хотя бы одного ребенка в семье увеличивает риск просрочки кредита;

- Состоящие в гражданском браке оказываются должниками чаще, чем люди, которые зарегистрировали официально свои отношения;

- Выплаты по кредитам за автомобиль и образование задерживают чаще, чем на другие цели.

**Таким образом, мы можем предоставить следующие рекомендации:**

- Самым оптимальным вариантом для получения кредита является женатый/замужний клиент, не имеющий детей, который берет кредит на покупку или ремонт квартиры и у которого высокий доход;

- Самым рискованным вариантом будет холостой человек, который берет кредит на автомобиль или образование и который имеет средний доход.

